In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Fack News Classification using LSTM
### It is binary Classification problem

In [33]:
import pandas as pd
import numpy as np
import os

In [34]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train_LSTM.csv')

In [35]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [36]:
# Drop Nan values
df = df.dropna()

In [37]:
# get independent features
X = df.drop('label', axis = 1)
y = df["label"]

In [38]:
X.shape

(18285, 4)

In [39]:
y.shape

(18285,)

In [40]:
import tensorflow as tf
tf.__version__

'2.12.0'

In [41]:
messages = X.copy()

In [42]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [43]:
messages.reset_index(inplace= True)

In [44]:
import nltk
import re
from nltk.corpus import stopwords

In [45]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [46]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
corpus = []
stemmer = PorterStemmer()
for i in range(0,len(messages)):
  sent = re.sub('[^a-zA-Z]]',' ',messages['title'][i])
  sent = sent.lower()
  sent = sent.split()
  sent = [stemmer.stem(word) for word in sent if not word in stopwords.words('english')]
  sent = ' '.join(sent)
  corpus.append(sent)

print(corpus)

['hous dem aide: didn’t even see comey’ letter jason chaffetz tweet', 'flynn: hillari clinton, big woman campu - breitbart', 'truth might get fire', '15 civilian kill singl us airstrik identifi', 'iranian woman jail fiction unpublish stori woman stone death adulteri', 'jacki mason: hollywood would love trump bomb north korea lack tran bathroom (exclus video) - breitbart', 'benoît hamon win french socialist party’ presidenti nomin - new york time', 'back-channel plan ukrain russia, courtesi trump associ - new york time', 'obama’ organ action partner soros-link ‘indivisible’ disrupt trump’ agenda', 'bbc comedi sketch "real housew isis" caus outrag', 'russian research discov secret nazi militari base ‘treasur hunter’ arctic [photos]', 'us offici see link trump russia', 're: yes, paid govern troll social media, blogs, forum websit', 'major leagu soccer, argentin find home success - new york time', 'well fargo chief abruptli step - new york time', 'anonym donor pay $2.5 million releas every

In [47]:
from tensorflow.keras.preprocessing.text  import one_hot
voc_size = 5000  # vocabulary size
onehot_repe = [one_hot(words,voc_size) for words in corpus]
onehot_repe

[[1259, 1241, 315, 535, 889, 3096, 2928, 230, 3976, 1026, 4207],
 [3783, 2775, 2082, 3962, 4828, 531, 3049],
 [4878, 1112, 2321, 4411],
 [903, 2144, 971, 799, 3111, 616, 4263],
 [1393, 4828, 273, 1243, 1211, 1652, 4828, 3918, 4622, 4570],
 [1220,
  230,
  2810,
  4002,
  2053,
  1184,
  266,
  1754,
  1353,
  4022,
  1206,
  2804,
  3113,
  3673,
  3049],
 [2383, 1010, 4819, 1314, 290, 4170, 4813, 1294, 1837, 777, 872],
 [2689, 3800, 467, 3916, 4235, 4249, 1184, 3472, 1837, 777, 872],
 [2919, 2113, 2705, 2329, 3480, 1025, 1775, 100, 3349, 4880],
 [717, 218, 2471, 1784, 2711, 730, 1471, 3571],
 [3410, 713, 1459, 2646, 840, 189, 4205, 597, 2580, 2070, 1424],
 [3111, 1432, 3096, 1025, 1184, 4235],
 [3562, 1055, 3315, 1565, 4168, 630, 1498, 128, 2571, 1784],
 [2471, 1391, 3046, 3861, 1564, 1263, 178, 1837, 777, 872],
 [3791, 1593, 4324, 4452, 4774, 1837, 777, 872],
 [4433, 990, 3579, 3424, 3243, 2006, 2204, 4903, 3836, 1911, 3594, 3098],
 [4652, 2334, 4647],
 [750, 1353, 1154, 559, 1184, 4

In [48]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
sent_length = 50  #making the length of the sentance same by pre - padding
embedded_doc = pad_sequences(onehot_repe, maxlen=sent_length,padding = 'pre')

In [49]:
print(embedded_doc)

[[   0    0    0 ... 3976 1026 4207]
 [   0    0    0 ... 4828  531 3049]
 [   0    0    0 ... 1112 2321 4411]
 ...
 [   0    0    0 ... 1837  777  872]
 [   0    0    0 ... 4970 3883 4870]
 [   0    0    0 ... 4802   16  891]]


In [50]:
embedded_doc[0]    # length of the sentace is 50

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0, 1259, 1241,  315,  535,  889,
       3096, 2928,  230, 3976, 1026, 4207], dtype=int32)

In [51]:
## create model
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential

embedding_vector_features = 50
model =  Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length = sent_length))
model.add(LSTM(150))
model.add(Dense(1,activation = 'sigmoid'))

model.compile(loss ='binary_crossentropy' ,optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 50, 50)            250000    
                                                                 
 lstm_1 (LSTM)               (None, 150)               120600    
                                                                 
 dense_1 (Dense)             (None, 1)                 151       
                                                                 
Total params: 370,751
Trainable params: 370,751
Non-trainable params: 0
_________________________________________________________________


In [52]:
# Dependent and indepentent features
X_input = np.array(embedded_doc)
y_input = np.array(y)

In [53]:
# Train Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_input,y_input,test_size = 0.33,random_state=42)

In [54]:
X_train.shape  # Shape of the X_train dataset

(12250, 50)

In [55]:
y_train.shape # Shape of the y_train dataset

(12250,)

In [56]:
# Training the model
model.fit(X_train, y_train, epochs = 10,batch_size=64,validation_data=(X_test,y_test))

Epoch 1/10
192/192 [==============================] - 45s 216ms/step - loss: 0.3063 - accuracy: 0.8536 - val_loss: 0.1989 - val_accuracy: 0.9176
Epoch 2/10
192/192 [==============================] - 38s 197ms/step - loss: 0.1221 - accuracy: 0.9527 - val_loss: 0.2064 - val_accuracy: 0.9185
Epoch 3/10
192/192 [==============================] - 37s 191ms/step - loss: 0.0907 - accuracy: 0.9679 - val_loss: 0.2299 - val_accuracy: 0.9120
Epoch 4/10
192/192 [==============================] - 35s 184ms/step - loss: 0.0519 - accuracy: 0.9827 - val_loss: 0.2836 - val_accuracy: 0.9104
Epoch 5/10
192/192 [==============================] - 37s 193ms/step - loss: 0.0249 - accuracy: 0.9927 - val_loss: 0.3509 - val_accuracy: 0.9122
Epoch 6/10
192/192 [==============================] - 36s 188ms/step - loss: 0.0126 - accuracy: 0.9965 - val_loss: 0.3668 - val_accuracy: 0.9074
Epoch 7/10
192/192 [==============================] - 36s 185ms/step - loss: 0.0069 - accuracy: 0.9981 - val_loss: 0.4625 - val_ac

In [57]:
# Prediction the model for X_test data set
y_pred = (model.predict(X_test)>0.5).astype("int64")
y_pred.shape

189/189 [==============================] - 8s 39ms/step


(6035, 1)

In [58]:
# Confusion Matrix for the model
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)

array([[3126,  293],
       [ 260, 2356]])

In [59]:
accuracy_score(y_pred,y_test)

0.9083678541839271




## Model have accuracy score for y_test data is 0.90836


